In [1]:

from sklearn.metrics import r2_score
import copy
import operator
from tensorflow.keras.losses import BinaryCrossentropy
import random
import pandas as pd
import keras.metrics as metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Flatten,Dense,Input,Reshape
from tensorflow.keras.models import Sequential
from random import randrange



In [2]:
from genetic_algorithm_net import GA

### Import dataset

In [3]:
df = pd.read_csv('../Data/processed_df.csv')

In [4]:
df.head()

,handgrip,weight_loss,exaustion,poor_endurance,gait_speed,balance,hearing_deficit,distance_vision,near_vision,depression_scale,...,idade_morador_8,idade_morador_9,idade_morador_10,idade_morador_11,idade_morador_12,idade_morador_13,idade_morador_14,adl_index,aadl_index,iadl_index
0,24.333333,0.0,1.0,1.0,3.500,4.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
1,25.666667,0.0,1.0,1.0,3.300,4.0,4.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,8.0
2,14.666667,NaN,2.0,1.0,4.595,4.0,2.0,3.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
3,24.000000,0.0,1.0,1.0,5.610,4.0,3.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,8.0
4,NaN,0.0,4.0,2.0,38.145,NaN,3.0,2.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,3.0


In [5]:
df.shape

(9412, 63)

### Remove missing data

In [6]:
ci_map = {
    'cognitive': ['semantic_memory', 'verbal_fluency', 'memory_recall', 'temporal_orientation'],
    'psychologicao': ['depression_scale', 'sleep_quality'],
    'locomotor': ['gait_speed', 'balance'],
    'vitality': ['handgrip', 'poor_endurance', 'weight_loss', 'exaustion'],
    'sensory': ['distance_vision', 'near_vision', 'hearing_deficit']
}

ci_cols = []
[ci_cols.extend(value) for value in ci_map.values()]
ci_cols = ci_cols + ['iadl_index']
ci_cols

['semantic_memory',
 'verbal_fluency',
 'memory_recall',
 'temporal_orientation',
 'depression_scale',
 'sleep_quality',
 'gait_speed',
 'balance',
 'handgrip',
 'poor_endurance',
 'weight_loss',
 'exaustion',
 'distance_vision',
 'near_vision',
 'hearing_deficit',
 'iadl_index']

In [7]:
df_clean = df.dropna(subset=ci_cols)
df_clean.shape

(6916, 63)

In [8]:
df_clean['iadl_index'].value_counts()

iadl_index
8.0    5860
7.0     659
6.0     212
5.0     101
4.0      46
3.0      25
2.0       6
0.0       4
1.0       3
Name: count, dtype: int64

In [9]:
# def calc_iadl(x):
#     if x == 8:
#         return 0
#     elif x == 7:
#         return 1
#     else:
#         return 2


def calc_iadl(x):
    if x == 8:
        return 0
    elif x == 7:
        return 1
    else:
        return 2

df_clean['iadl_index'] = df_clean['iadl_index'].map(lambda x: calc_iadl(x))

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_9916\3209756502.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['iadl_index'] = df_clean['iadl_index'].map(lambda x: calc_iadl(x))


In [10]:
df_clean['iadl_index'].value_counts()

iadl_index
0    5860
1     659
2     397
Name: count, dtype: int64

In [11]:
df_clean['iadl_index']


0       0
1       0
3       0
5       2
7       2
       ..
9403    0
9404    0
9405    0
9407    0
9411    0
Name: iadl_index, Length: 6916, dtype: int64

### Split Train and test


In [12]:
import pickle
# Define dataset
df_autoencoder = df_clean[ci_cols]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_autoencoder.drop(columns=['iadl_index']), df_autoencoder['iadl_index'], test_size=0.2, random_state=42)

In [13]:
X_train.shape, y_test.shape

((5532, 15), (1384,))

In [14]:

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)




with open('./out/autoencoder_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
X_train_scaled.shape, X_test_scaled.shape

((5532, 15), (1384, 15))

In [15]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=42, sampling_strategy = 'all')
X_train_scaled_over, y_train_over = ros.fit_resample(X_train_scaled, y_train)

#### Run GA

In [16]:
DNA_parameter = [["tanh","softmax","relu","sigmoid","linear"],
                 ["sgd","adam"]]



In [17]:
ga = GA(coding_size=2, X_train=X_train_scaled_over, X_test=[], y_train=y_train_over, y_test=[], DNA_parameter=DNA_parameter, epochs=10)

In [18]:
y_train_over.value_counts()

iadl_index
0    4681
2    4681
1    4681
Name: count, dtype: int64

In [19]:
pop = ga.create_population(hidden_layers_encoder=2, hidden_layers_decoder=3, max_dim_increase=10, population_size=20)

AttributeError: 'GA' object has no attribute 'num_classes'

In [ ]:
ga.print_architecture(pop[5])

------------------------------
15 None
10 linear
3 tanh
2 softmax
1 sigmoid
optimizer sgd
------------------------------


In [ ]:
ga.train(ga.build_neural_net(pop[5]))

InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "C:\Users\mwuillau\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\mwuillau\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\mwuillau\AppDa

In [ ]:
best_arch = ga.GA(population=pop, n_generations=2)

Generation: 0


c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6216 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6099 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6270 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6241 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6426 - loss: 0.0000e+00


c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6376 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6319 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6283 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6248 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6340 - loss: 0.0000e+00


c:\Users\mwuillau\Documents\IC\intrinsic-capacity\myenv\lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6278 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6177 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6154 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6229 - loss: 0.0000e+00
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6370 - loss: 0.0000e+00


IndexError: tuple index out of range

### Import Genetic Result

In [ ]:
model = tf.keras.models.load_model('./out/best_model.h5')

In [ ]:
model.predict(X_test_scaled)

32/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


array([[-0.2995344 , -0.41704544, -0.3759935 , ...,  0.19974044,
         0.05309695,  0.05122371],
       [-0.54145867, -0.43984187, -0.53072876, ...,  0.88966703,
         0.8229447 ,  0.57867724],
       [-0.38256487, -0.39333206, -0.35648096, ...,  0.77078557,
         0.67072767,  0.5331141 ],
       ...,
       [ 0.73997784,  0.7839768 ,  0.79190093, ..., -0.09436029,
         0.05686268, -0.16890971],
       [-0.1041927 , -0.20652618, -0.01900695, ..., -0.10671389,
        -0.18163204,  0.04037876],
       [-0.05616935, -0.08047217, -0.03674935, ..., -0.20170137,
        -0.27207446, -0.21064   ]], dtype=float32)

In [ ]:
from sklearn.metrics import r2_score

decoded_data = model.predict(X_test_scaled)
global_r2 = r2_score(X_test_scaled, decoded_data)
print(global_r2)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
0.38231258232286514
